In [1]:
import os

from IPython.display import display, Markdown, Image

In [2]:
REPO_PATH = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
TASK_PATH = os.path.join(REPO_PATH, "tasks", "10-vectors.md")
DATA_PATH = os.path.join(os.getcwd(), "1551.gov.ua", "raw")

In [3]:
def show_markdown(path):
    with open(path, 'r') as fh:
        content = fh.read()
    display(Markdown(content))

In [4]:
show_markdown(TASK_PATH)

# Векторні представлення

## Завдання 1

Побудуйте покращений класифікатор, який буде предбачати категорію запиту до служби 1551 з використанням тільки векторів слів (не використовувати інші ознаки, такі як самі слова, нграми і т.д) для всіх категорій.

У якості бейзлайну використайте класифікатор kNN, який ви побудували на практичному.

Зробіть як мінімум 2 ітерації покращення, які можуть полягати у:

- підборі найкращого варіанту векторів з наведених тут: https://lang.org.ua/en/models/#anchor4 або інших (fasttext, Tf-IDF, LDA, ...) Ви навіть можете дотренувати вектори на власній тренувальній множині. Можете також спробувати агрегацію декількох векторів.
- будь-яких видах передпроцесингу і попередньої обробки (фільтрація мови, стопслів, службових частин мови, лематизація, нормалізація, ...)
- побудови векторів документу з використанням таких підходів як pargaraph vectors (doc2vec), ElMo, Universal Sentece Encoder...
- використання довільних алгоритмів класифікації та ансамблів цих алгоритмів
- групуванні категорій (на основі кластеризації або якихось евристик) та побудови багаторівневих класифікаторів

Ваша мета: досягнути такого результату по якості, щоб було не соромно дивитись в очі Кличкові. ;)

## Завдання 2

Візуалізуйте ваші класи з використанням t-SNE. Якщо ви будете використовувати агрегацію категорій, то можете візуалізувати тільки категорії вищого рівня і, для прикладу, одну з них. Якщо ви не будете групувати категорії, то використовуйте для візуалізації найбільш частотні (30-50). Спробуйте досягти достатньо розділеної картинки, експериментуючи з параметрами візуалізації (звісно, якщо ваші вхідні дані дозволять вам це).


## Оцінювання

Оцінка:
- 90 балів за класифікатор (рішення, які будуть мати якість більше 0.8 за macro_average або більше 0.9 за micro_average отримають додаткові 10 балів)
- 10 балів за візуалізацію

Крайній термін: 16.05.2020


## Read data

In [5]:
import re
import gzip
import json

import pandas as pd
import numpy as np

from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
from pathlib import Path
from collections import Counter

In [6]:
import pycld2 as cld2
from langdetect import detect, lang_detect_exception

def lang_detect(x):
    try:
        return detect(x.lower())
    except lang_detect_exception.LangDetectException:
        return ''

def lang_detect_v2(x):
    try:
        x = ''.join(_ for _ in " ".join(x.split()[:100]).lower() if _.isprintable())
        isReliable, textBytesFound, details = cld2.detect(x)
        res = sorted(details, key=lambda x: x[2], reverse=True)[0][1]
        if res == 'un':
            res = detect(x.lower())
        return res
    except:
        return ""

In [7]:
res = []
for file_path in tqdm(Path(DATA_PATH).rglob('*.gz')):
    with gzip.open(file_path, 'rt', encoding='utf8') as zipfile:
        obj = json.load(zipfile)[0]
    res.append(obj)
df = pd.DataFrame(res)

127329it [00:18, 6964.48it/s]


In [8]:
df['manager_name'] = df['CallZManager'].map(lambda x: "" if not re.search(r"\[([\s\w\.]+)\]", x) 
                                                         else re.findall(r"\[([\s\w\.]+)\]", x)[0].strip())
df['entity_name'] = df['CallZManager'].map(lambda x: re.sub(r"\[([\s\w\.]+)\]", "", x))
df['cat_count'] = df['CallZType'].map(df['CallZType'].value_counts().to_dict())
df['word_count'] = df['CallZText'].str.split().map(len)

In [9]:
%%time

lst = process_map(lang_detect, df['CallZText'].values, chunksize=1, max_workers=8)
df['lang'] = lst


CPU times: user 30.2 s, sys: 3.63 s, total: 33.8 s
Wall time: 2min 12s


In [10]:
df['word_count'].describe()

count    127329.000000
mean         63.134384
std          59.138254
min           0.000000
25%          22.000000
50%          45.000000
75%          84.000000
max         418.000000
Name: word_count, dtype: float64

In [11]:
print(f"Records with > 100 label samples: {df.loc[df.cat_count > 100].shape[0] / df.shape[0] * 100:.2f} %")

Records with > 100 label samples: 89.95 %


In [12]:
df['lang'].value_counts()

uk    69345
ru    56944
        540
bg      319
mk      167
pl        4
en        2
et        2
ca        2
sw        1
fr        1
sl        1
sk        1
Name: lang, dtype: int64

In [13]:
# Counter(" ".join(df['entity_name'].values).lower().split()).most_common(10)

In [14]:
data_uk = df.loc[(df.cat_count > 100) & (df.lang.isin(['uk']))]#[['CallZText', 'CallZType']]
data_ru = df.loc[(df.cat_count > 100) & (df.lang.isin(['ru']))]#[['CallZText', 'CallZType']]
data_uk = data_uk.rename(columns=dict(zip(['CallZText', 'CallZType'], ['text', 'label'])))
data_ru = data_ru.rename(columns=dict(zip(['CallZText', 'CallZType'], ['text', 'label'])))
print(data_uk.shape, data_ru.shape)

(61999, 25) (51648, 25)


In [15]:
data_uk.head()

,StatusTitle,CallZCdate,CallZReply,text,CallZManager,label,CallDate,CallCode,FbId,Enabled,...,PY,showStatus,files,feedfiles,reply,manager_name,entity_name,cat_count,word_count,lang
0,Виконано,2015-07-03 00:00:00,,Добрий день! Відповідно до контрольної картки ...,КП ”УЗН” [Сінцов Геннадій Львович ],Утримання парків та лісопарків у місті Києві,2015-06-18 13:20:58,І-2513,149382,1,...,30.636674,,"[{'id': '81573', 'InOut': '0', 'file': '/feedb...","[{'id': '49925', 'file': '/feedback/fdocs/3201...","{'1436176081': {'id': '103381', 'callid': '320...",Сінцов Геннадій Львович,КП ”УЗН”,399,82,uk
2,Виконано,2013-10-21 00:00:00,,В будинку 29 на вулиці Якуба Коласа відсутнє о...,КП РЕО-5 [Бондар Юрій Іванович],Відсутність опалення,2013-10-17 18:20:15,І-4646,28337,1,...,30.383763,,,,[],Бондар Юрій Іванович,КП РЕО-5,6020,55,uk
3,На доопрацюванні виконавцем,2014-01-28 00:00:00,,"Доброго дня, вимушена звертатися за допомогою ...",ПАТ ”Київенерго” - ТРМ [Фоменко Олександр Вале...,Недостатній тиск ГВП,2014-01-13 14:30:50,І-83,40156,1,...,30.414153,,,"[{'id': '12021', 'file': '/feedback/fdocs/2690...","{'1390476591': {'id': '23506', 'callid': '2690...",Фоменко Олександр Валерійович,ПАТ ”Київенерго” - ТРМ,303,97,uk
6,В роботі,2015-05-27 00:00:00,,Відсутне ГВП (в об’яві - на невизначений термін),ПАТ ”Київенерго” - ЖТЕ [Фоменко Олександр Вале...,Відсутність ГВП,2015-05-12 17:43:50,І-1871,136934,1,...,30.432559,,,,[],Фоменко Олександр Валерійович,ПАТ ”Київенерго” - ЖТЕ,13273,8,uk
8,Чекає на перевіку контр.відділом,2014-10-06 00:00:00,,Доброго дня! Вже неможливо терпіти відсутність...,ПАТ ”Київенерго” - ТРМ [Фоменко Олександр Вале...,Відсутність ГВП,2014-09-21 12:10:28,І-2631,74345,1,...,30.63972,,,"[{'id': '21895', 'file': '/feedback/fdocs/2875...",[],Фоменко Олександр Валерійович,ПАТ ”Київенерго” - ТРМ,13273,66,uk


In [16]:
res = df.CallZType.value_counts()

for k, v in res.items():
    if "електро" in k.lower():
        print(v, k)

print('\n****************\n')

for k, v in res.items():
    if "паркув" in k.lower():
        print(v, k)
        
print('\n****************\n')

for k, v in res.items():
    if "опал" in k.lower():
        print(v, k)

print('\n****************\n')

for k, v in res.items():
    if "гвп" in k.lower():
        print(v, k)

print('\n****************\n')

for k, v in res.items():
    if "хвп" in k.lower():
        print(v, k)

565 Відсутнє електропостачання
553 Ремонт та заміна електроприладів
514 Перебої електропостачання
65 Підключення МАФ до комунікацій (електро -, водопостачання, тощо)
62 Встановлення та експлуатація електролічильників
50 Питання нарахування плати за електропостачання (розрахунки за електропостачання)
36 Перерахунок, нарахування плати та боргу  за електропостачання
22 Питання, що стосуються лічильників на електроенергію
19 Відключення електропостачання за борги
14 Питання стосовно нарахування боргу за електропостачання
13 Укладання договорів на постачання електроенергії
8 Перерахунок плати за відсутність електропостачання
4 Відключення за борги електропостачання
1 Про впровадження абонентських книжок АК ”Київенерго”,на оплату за електроенергію

****************

616 Паркування авто у місцях загального користування
359 Паркування авто на прибудинковій території
218 Розміщення паркувальних майданчиків
190 Зберігання транспортних засобів, порушення правил паркування
105 Паркування на зелені

## Preprocessing

In [17]:
import stanfordnlp
import warnings

warnings.filterwarnings("ignore")

In [18]:
nlp_uk = stanfordnlp.Pipeline(lang='uk')
nlp_ru = stanfordnlp.Pipeline(lang='ru')

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu_tokenizer.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu_tagger.pt', 'pretrain_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu.pretrain.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu_lemmatizer.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu_parser.pt', 'pretrain_path': '/home/dima/stanfordnlp_resources/uk_iu_models/uk_iu.pretrain.pt', 'lang': '

In [19]:
def clean_text(x, words_num=100):
    x = ''.join(_ for _ in " ".join(filter(None, x.split()[:words_num])).lower() if _.isprintable())
    x = x.encode('utf-8', "ignore").decode("utf-8")
    
    x = x.replace("”", "’")
    for punct in ['!', ".", "?", ","]:
        x = re.sub(r"\{}+".format(punct), punct, x)
    return x


def text2tokens(text, lang='uk', lemma=False, ignore_punct=True):
    nlp_map = {'uk': nlp_uk, 'ru': nlp_ru}
    nlp = nlp_map.get(lang)
    tokens = []
    
    for sentence in nlp(text).sentences:
        for token in sentence.words:
            if ignore_punct and token.upos == 'PUNCT':
                continue
            if lemma:
                tokens.append(token.lemma)
            else:
#                 tokens.append((token.text, token.upos))
                tokens.append(token.text)
                
    return tokens

In [20]:
sample = data_uk.sample(3)

In [21]:
for item in sample.text.values:
    print(item)
    print("*********")

Надсилаючи письмові відповіді на мої звернення, в КП ”Плесо” склали 4 арк. з відповідями в 1 конверт. П’ятий акруш, як виявилося, відповідь НЕ на моє звернення П-10092/1 від 13.11.2013 р. (додаю фото аркуша). Мабуть це сталося помилково. Повідомляю про це та прошу переадресувати відповідь заявникові.
*********
відновіть освітлення з першого по 10-й поверх, де воно передбачено з моменту забудови будинку, працівники ЖЕКу його обрізали що б не вкручувати лампочки!!! на сходових прольотах, і така ситуація на кожному поверсі. Верніть патрони та вкрутіть лампочки, адже там передбачено освітлення проектом!!! на всіх поверхах тільки провода стирчать. Чому ви закриваєте звернення нічого не зобивши не розумію.... освітлення біля не помитих вікон...... там тільки провода стирчать. Свободи, 46, 3-й підїзд
*********
Ми проживаємо в будинку по вул.Леніна 61 з 2003 року. З часу заселення в під’їздах жодних ремонтних робіт не проводилося. Хоча в сусідньому будинку (буд 59), який заселений пізніше нашо

In [22]:
for item in sample.text.values:
    print(clean_text(item))
    print("*********")

надсилаючи письмові відповіді на мої звернення, в кп ’плесо’ склали 4 арк. з відповідями в 1 конверт. п’ятий акруш, як виявилося, відповідь не на моє звернення п-10092/1 від 13.11.2013 р. (додаю фото аркуша). мабуть це сталося помилково. повідомляю про це та прошу переадресувати відповідь заявникові.
*********
відновіть освітлення з першого по 10-й поверх, де воно передбачено з моменту забудови будинку, працівники жеку його обрізали що б не вкручувати лампочки! на сходових прольотах, і така ситуація на кожному поверсі. верніть патрони та вкрутіть лампочки, адже там передбачено освітлення проектом! на всіх поверхах тільки провода стирчать. чому ви закриваєте звернення нічого не зобивши не розумію. освітлення біля не помитих вікон. там тільки провода стирчать. свободи, 46, 3-й підїзд
*********
ми проживаємо в будинку по вул.леніна 61 з 2003 року. з часу заселення в під’їздах жодних ремонтних робіт не проводилося. хоча в сусідньому будинку (буд 59), який заселений пізніше нашого, в першом

In [23]:
# text2tokens(item)

In [24]:
data_uk['text_clean'] = data_uk['text'].map(clean_text)
data_ru['text_clean'] = data_ru['text'].map(clean_text)

## Embeddings

### Universal Sentence Encoder

In [25]:
import tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub

from scipy import spatial

In [26]:
tf.__version__

'2.0.1'

In [27]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

True

In [28]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
use_model = hub.load(module_url)

In [29]:
def cos_sim(vec1, vec2):
    return 1 - spatial.distance.cosine(vec1, vec2)

In [30]:
def use_embed(text, numpy=True):
    res = np.zeros((1,512))
    if isinstance(text, str):
        text = [text]
    try:
        res = use_model(text)
        if numpy:
            res = res.numpy()
    except Exception as e:
        print(e, text)
    return res

In [31]:
lang = 'ru'

if lang == 'ru':
    tmp = data_ru.loc[data_ru.cat_count == data_ru.cat_count.max()].sample(2)
else:
    tmp = data_uk.loc[data_uk.cat_count == data_uk.cat_count.max()].sample(2)
    
cat = tmp['label'].unique()[0]
sample1 = clean_text(tmp['text'].values[0])
sample2 = clean_text(tmp['text'].values[1])

print(cat, sample1, sample2, sep="\n*************\n")

Відсутність ГВП
*************
здравствуйте. почему плановое отключение горячей воды, в нашем доме, никогда не заканчивается вовремя. я здесь живу много лет и ни одного раза киевэнерго не вложилось в обещанные 2 недели, всегда минимум месяц дом без горячей воды. вот и сейчас воду должны были дать 1 июля, но её нет! автоответчик киевэнерго говорит, что в соломенском районе нет аварийных ситуаций, а автоответчик кп ’батыевское’ обещает воду не раньше 9 июля! они даже в своей лжи синхронизироваться не могут. я считаю данную ситуацию возмутительной! нельзя столько лет врать людям о двухнедельном отключении горячей воды. надеюсь на вашу помощь в решении данного вопроса и
*************
до сих пор в нашем доме нет горячего водоснабжения. но еще периодически отключают и холодную воду в связи с производством каких- то ремонтных работ вокруг нашего дома. при этом вдоль дороги выливается огромное количество воды, которая стоит сейчас немалые деньги. наш дом окопан был уже, по-моему, со всех сторон

In [32]:
print('Embedding shape:', use_embed(sample1).shape)
print('Text similarity:', cos_sim(use_embed(sample1), use_embed(sample2)))

Embedding shape: (1, 512)
Text similarity: 0.6137630939483643


I'm pleasantly surprised that Russian USE model somehow performs on Ukrainian

### Word2vec / Glove

Ubercorpus embeddings

In [33]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.keyedvectors import KeyedVectors

In [34]:
# glove2word2vec(glove_input_file="ubercorpus.lowercased.tokenized.glove.300d", 
#                word2vec_output_file="ubercorpus.lowercased.tokenized.glove.300d.txt")

In [35]:
%%time

glove_model = KeyedVectors.load_word2vec_format("ubercorpus.lowercased.tokenized.glove.300d.txt", binary=False)

CPU times: user 1min 40s, sys: 734 ms, total: 1min 41s
Wall time: 1min 40s


In [36]:
%%time

w2v_model = KeyedVectors.load_word2vec_format("ubercorpus.lowercased.tokenized.word2vec.300d", binary=False)

CPU times: user 1min 39s, sys: 722 ms, total: 1min 40s
Wall time: 1min 39s


In [37]:
%%time

lexvec_model = KeyedVectors.load_word2vec_format("ubercorpus.lowercased.tokenized.300d", binary=False)

CPU times: user 1min 39s, sys: 800 ms, total: 1min 40s
Wall time: 1min 39s


In [38]:
w2v_model.wv.most_similar('москаль', topn=5)

[('корбан', 0.703376054763794),
 ('кернес', 0.6943342685699463),
 ('задирко', 0.6910302639007568),
 ('мінаєв', 0.6881823539733887),
 ('іванущенко', 0.676155686378479)]

In [39]:
glove_model.wv.most_similar('москаль', topn=5)

[('геннадій', 0.5712622404098511),
 ('нардеп', 0.5634081959724426),
 ('москаля', 0.5238221883773804),
 ('луценко', 0.5117676258087158),
 ('депутат', 0.49746325612068176)]

In [40]:
lexvec_model.wv.most_similar('москаль', topn=5)

[('кернес', 0.6542044878005981),
 ('жебрівський', 0.6423298120498657),
 ('тука', 0.6347901225090027),
 ('зубко', 0.6334406733512878),
 ('корбан', 0.6276117563247681)]

as lowercased model trained on news, wiki and literature, by ``москаль`` it understands ``Генадій Москаль``, but not that dirty ``москаль`` from Mordor. It happens because politician appeared much more frequently in train corpus that the second one. But in my case I can't use cased model, because there are lots of reviews among train data with capsed text and if I will not lowercase them, they will add lots of noise to my data.

In [41]:
def gensim_embed(tokens, model=glove_model):
    """ average all vectors that are in vocab, ignore OOV """
    res = []
    for token in tokens:
        if token in w2v_model.vocab:
            res.append(model.wv.__getitem__([token]))
    
    return np.mean(np.vstack(res), axis=0).reshape(1, -1)

In [42]:
lang = 'uk'

if lang == 'ru':
    tmp = data_ru.loc[data_ru.cat_count == data_ru.cat_count.max()].sample(2)
else:
    tmp = data_uk.loc[data_uk.cat_count == data_uk.cat_count.max()].sample(2)
    
cat = tmp['label'].unique()[0]
sample1 = clean_text(tmp['text'].values[0])
sample2 = clean_text(tmp['text'].values[1])

print(cat, sample1, sample2, sep="\n*************\n")

Відсутність ГВП
*************
на мое звернення г-9259 я досі не отримала конкретної відповіді, хоча термін давно сплив. з 4 червня пат ’київенерго’ припинило виконання своїх зобов’язаннь по гвп через заборгованість державного бюджету. 7 червня домовленість між міською владою и пат ’києвенерго’ була досягнута и 8 червня було обіцяно відновлення постачання гвп. однак у той же день 8 червня начебто був виявлений прорив теплотраси за адресою вул. акад. туполєва,16-д. діагностика прориву тривала аж до 12 червня. з 12 червня по 14 червня начебто тривали ремонтні роботи. однак 14 червня замість гвп я, як споживач почула нову версію проблеми гвп- новий прорив и ремонтні роботи
*************
сьогодні без попередження відключили гарячу воду. у т.зв. ’диспетчерській службі оболонського району’ нічого про це не знають. по будинку нічого не заплановано - так прокоментували у цій службі. скарга - чому (знову!) немає попереджень про відключення води?


In [43]:
print('Embedding shape:', gensim_embed(text2tokens(sample1)).shape)
print('Text similarity w2v:', cos_sim(gensim_embed(text2tokens(sample1), model=w2v_model), 
                                      gensim_embed(text2tokens(sample2), model=w2v_model)))
print('Text similarity glove:', cos_sim(gensim_embed(text2tokens(sample1), model=glove_model), 
                                        gensim_embed(text2tokens(sample2), model=glove_model)))
print('Text similarity lexvec:', cos_sim(gensim_embed(text2tokens(sample1), model=lexvec_model), 
                                        gensim_embed(text2tokens(sample2), model=lexvec_model)))

Embedding shape: (1, 300)
Text similarity w2v: 0.38542604446411133
Text similarity glove: 0.8393498063087463
Text similarity lexvec: 0.6127337217330933


2 comments:
* ukrainian models performs well both on uk and ru
* as I expected, glove works slightly better in most of cases

### BERT

In [44]:
BERT_PRETRAINED_DIR = 'multi_cased_bert_base_uk'

VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')

In [45]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# import bert

#### tokenize

In [46]:
tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_DIR)
# tokenizer = bert.tokenization.bert_tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=False)

In [47]:
text = "вже майже добу відсутня гаряча вода"
marked_text = "[CLS] " + text + " [SEP]"

tokenized_text = tokenizer.tokenize(marked_text)

print(tokenized_text)

['[CLS]', 'вже', 'ма', '##иже', 'до', '##бу', 'від', '##су', '##тня', 'га', '##ря', '##ча', 'вода', '[SEP]']


In [48]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
вже          23,692
ма           97,744
##иже        94,152
до           10,344
##бу         19,590
від          11,141
##су         16,417
##тня        80,897
га           16,616
##ря         39,025
##ча         13,035
вода         30,953
[SEP]           102


I don't know why, but TF tokenizer works not correctly and doesn't recognize [CLS] and [SEP] tokens

#### load model

In [49]:
# model = BertModel.from_pretrained(BERT_PRETRAINED_DIR)

# model.eval()

model was trained with tf instead of pytorch, but loads not correctly, so I decided to work on this problem later

In [50]:
# bert_params = bert.params_from_pretrained_ckpt(BERT_PRETRAINED_DIR)
# l_bert = bert.BertModelLayer.from_params(bert_params, name="bert")

### Finalize and train/test split

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
data = pd.concat([data_ru, data_uk])[['text_clean', 'label']]
target = data['label']

In [53]:
RANDOM_SEED = 0
num_classes = target.unique().shape[0]

In [54]:
res = []

for i in tqdm(range(int(data.shape[0] / 100) + 1)):
    res.append(use_embed(data['text_clean'].values[i*100:(i+1)*100]))

x_vec = np.vstack(res)

100%|██████████| 1137/1137 [00:46<00:00, 24.60it/s]


In [55]:
x_vec.shape

(113647, 512)

In [56]:
# res = []

# for item in tqdm(data_uk['text_clean'].values):
#     res.append(gensim_embed(text2tokens(item), model=glove_model))

In [57]:
X_train, X_test, y_train, y_test = train_test_split(x_vec, target.values, 
                                                    test_size=0.2, 
                                                    random_state=RANDOM_SEED,
                                                    stratify=target)

In [58]:
print(X_train.shape, X_test.shape)

(90917, 512) (22730, 512)


## Modeling

### Baseline

In [59]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *

In [60]:
%%time

knn_clf = KNeighborsClassifier(n_neighbors=3, metric='cosine')

knn_clf.fit(X_train, y_train)

CPU times: user 278 ms, sys: 0 ns, total: 278 ms
Wall time: 277 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [61]:
pred = knn_clf.predict(X_test)

print(classification_report(pred, y_test))

                                                                                  precision    recall  f1-score   support

                                                                     Інші Подяки       0.24      0.24      0.24       102
                                              Інші технічні недоліки стану ліфту       0.55      0.28      0.37       568
                Аварійний, травмонебезпечний стан утримання об’єктів благоустрою       0.19      0.14      0.16       124
                                          Бажаючі отримати ”Картки киянина (КК)”       0.77      0.45      0.57        38
                              Будівництво /дооблаштування спортивних майданчиків       0.53      0.29      0.37        59
                                                                 Будівництво АЗС       0.79      0.45      0.58        51
                                                        Будівництво в нічний час       0.50      0.23      0.32        47
                       

### Simple NNs

In [62]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (Input, Dense, Dropout, Flatten, Reshape, concatenate, BatchNormalization, 
                                     Conv1D, MaxPooling2D, Embedding, Convolution1D, MaxPool1D, 
                                     MaxPooling1D, GlobalMaxPool1D, Lambda,
                                     LSTM, Bidirectional, GRU, SimpleRNN)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [63]:
# from tensorflow.keras import models, layers, preprocessing, callbacks

In [65]:
def create_model(name, max_sequence_len,
                 lr = 1e-3,
                 activation_cnn = 'relu',
                 activation_final = 'softmax',
                 init_mode='uniform'):
    
    sequence_input = Input(shape=(max_sequence_len, 1))
    embed = Dropout(rate=0.2)(sequence_input)
    cnn1 = Convolution1D(80, 3, padding='same', strides=1, activation=activation_cnn, kernel_initializer=init_mode)(embed)
    cnn1 = MaxPool1D(pool_size=4)(cnn1)
    cnn2 = Convolution1D(50, 5, padding='same', strides=1, activation=activation_cnn, kernel_initializer=init_mode)(embed)
    cnn2 = MaxPool1D(pool_size=4)(cnn2)
    cnn3 = Convolution1D(50, 10, padding='same', strides=1, activation=activation_cnn, kernel_initializer=init_mode)(embed)
    cnn3 = MaxPool1D(pool_size=4)(cnn3)
#     cnn = concatenate([cnn1, cnn2, cnn3], axis=-1)
    flat = GlobalMaxPool1D()(cnn1)
    drop = Dropout(rate=0.5)(flat)
    output = Dense(num_classes, activation=activation_final, kernel_initializer=init_mode)(drop)
    model = Model(inputs=sequence_input, outputs=output, name=name)
    
    optimizer = Adam(lr=lr)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['categorical_accuracy'])
    #model.compile(optimizer=optimizer, loss='kullback_leibler_divergence', metrics=['categorical_accuracy'])
    
    return model

In [66]:
model = create_model(name='cnn_model', max_sequence_len=X_train.shape[1])

callbacks = [
    ReduceLROnPlateau(patience=3, factor=0.2, verbose=True), 
    EarlyStopping(patience=5, verbose=True), 
    ModelCheckpoint(filepath='model_cnn.h5', save_best_only=True, verbose=True)
]

model.summary()

Model: "cnn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 1)]          0         
_________________________________________________________________
dropout (Dropout)            (None, 512, 1)            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 512, 80)           320       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 128, 80)           0         
_________________________________________________________________
global_max_pooling1d (Global (None, 80)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense (Dense)                (None, 186)               15

In [ ]:
history = model.fit(X_train.reshape(-1, 512, 1), y_train,
                    class_weight='balanced',
                    epochs=1,
                    batch_size=16,
#                     callbacks=callbacks,
                    validation_split=0.1,
                    verbose=1)

Train on 81825 samples, validate on 9092 samples


In [ ]:
pred = history.predict(X_test)

print(classification_report(pred, y_test))